In [ ]:
# import yfinance as yf
# from backtesting.lib import crossover

# # 0. Config 
# TICKER = "AAPL"
# START_DATE = "2019-01-01"

# # 1. Data 
# data = yf.download(TICKER, start=START_DATE, progress=False)[
#             ["Open", "High", "Low", "Close", "Volume"]
#         ].dropna().droplevel(1, axis=1)  # remove multi-index
# data.index.name = "Date"

# # 2. Petit indicateur : SMA20
# # def SMA(s, n=20): return s.rolling(n).mean()
# def SMA(values, n):
#     """
#     Return simple moving average of `values`, at
#     each step taking into account `n` previous values.
#     """
#     return pd.Series(values).rolling(n).mean()

# class MultiTradeHold5(Strategy):
#     # --- paramètres modifiables -------
#     hold_bars = 5
#     sma_len   = 20
#     # -----------------------------------

#     def init(self):
#         self.sma = self.I(SMA, self.data.Close, self.sma_len)

#     def next(self):
#         # ❶ Fermer tous les trades qui ont 5 barres d'âge
#         for trade in list(self.trades): # toutes positions ouvertes
#             if (len(self.data) - trade.entry_bar) > self.hold_bars:
#                 trade.close()

#         # ❷ Signal d'entrée : cross Close ↗ SMA20  (long uniquement pour l'exemple)
#         if crossover(self.data.Close, self.sma):
#             # aucune vérification self.position → on empile librement
#             self.buy(size=1)

# # 3. Lancement du back-test
# bt = Backtest(data, MultiTradeHold5,
#               cash=10_000, commission=0.002, trade_on_close=True)
# stats = bt.run()

# # Visualize the backtest results with a plot
# # Set a custom size and save to HTML file for future reference
# bt.plot(plot_width=1200, filename="apple_backtest_results.html")


In [1]:
from backtesting import Backtest, Strategy
import pandas as pd

def RSI(values, n=14):
    """
    Calculate the Relative Strength Index for a given price series
    """
    # Calculate price changes
    delta = pd.Series(values).diff()
    # Split gains (up) and losses (down)
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    # Calculate EMA of up and down
    roll_up = up.rolling(n).mean()
    roll_down = down.rolling(n).mean()
    # Calculate RS and RSI
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

class RSIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5  # Par défaut comme dans l'exemple
    rsi_period = 14
    rsi_buy_threshold = 40
    # -----------------------------------

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Signal d'entrée : RSI < 40 (survendu)
        if self.rsi[-1] < self.rsi_buy_threshold:
            self.buy(size=1)

def BBANDS(values, n=20, dev=2):
    """
    Calculate Bollinger Bands for a price series
    Returns: middle band, upper band, lower band, and percent B
    """
    sma = pd.Series(values).rolling(n).mean()
    std = pd.Series(values).rolling(n).std(ddof=0)
    upper_band = sma + (std * dev)
    lower_band = sma - (std * dev)
    # Calculate %B (percent bandwidth)
    percent_b = (values - lower_band) / (upper_band - lower_band)
    
    return sma, upper_band, lower_band, percent_b

class BollingerStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    bb_period = 20
    bb_dev = 2
    # -----------------------------------

    def init(self):
        self.sma, self.upper, self.lower, self.percent_b = self.I(BBANDS, self.data.Close, self.bb_period, self.bb_dev)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand le prix est à l'intérieur des bandes (0 ≤ pctB ≤ 1)
        if 0 <= self.percent_b[-1] <= 1:
            self.buy(size=1)
        
        # Vendre quand le prix est au-dessus de la bande supérieure (pctB > 1)
        elif self.percent_b[-1] > 1:
            self.sell(size=1)

def CMF(high, low, close, volume, n=20):
    """
    Calculate Chaikin Money Flow
    """
    money_flow_multiplier = ((close - low) - (high - close)) / (high - low)
    money_flow_volume = money_flow_multiplier * volume
    cmf = money_flow_volume.rolling(n).sum() / volume.rolling(n).sum()
    return cmf

class ChaikinMoneyFlowStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    cmf_period = 20
    cmf_buy_threshold = 0.20
    cmf_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.cmf = self.I(CMF, self.data.High, self.data.Low, self.data.Close, self.data.Volume, self.cmf_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand CMF > 0.20
        if self.cmf[-1] > self.cmf_buy_threshold:
            self.buy(size=1)
        
        # Vendre quand CMF < -0.20
        elif self.cmf[-1] < self.cmf_sell_threshold:
            self.sell(size=1)

def Stochastic_R(high, low, close, n=14):
    """
    Calculate Williams %R (similar to Lane Stochastic)
    """
    highest_high = pd.Series(high).rolling(n).max()
    lowest_low = pd.Series(low).rolling(n).min()
    percent_r = (highest_high - close) / (highest_high - lowest_low)
    return percent_r

class StochasticRStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    stoch_period = 14
    buy_threshold = 0.50
    sell_threshold = 0.90
    # -----------------------------------

    def init(self):
        self.percent_r = self.I(Stochastic_R, self.data.High, self.data.Low, self.data.Close, self.stoch_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand %R < 0.50
        if self.percent_r[-1] < self.buy_threshold:
            self.buy(size=1)
        
        # Vendre quand %R > 0.90
        elif self.percent_r[-1] > self.sell_threshold:
            self.sell(size=1)

def TDI_DI(values, n=14):
    """
    Calculate Trend Detection Index (TDI) and Direction Index (DI)
    """
    # Calcul simplifié - TDI est la pente du prix, DI est la dérivée seconde
    price = pd.Series(values)
    tdi = price.diff(n)  # Différence sur n périodes = momentum
    di = tdi.diff(1)     # Dérivée du momentum = accélération
    return tdi, di

class TDIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    tdi_period = 14
    # -----------------------------------

    def init(self):
        self.tdi, self.di = self.I(TDI_DI, self.data.Close, self.tdi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand TDI > 0 et DI > 0 (momentum croissant)
        if self.tdi[-1] > 0 and self.di[-1] > 0:
            self.buy(size=1)
        
        # Vendre quand TDI > 0 et DI < 0 (perte de momentum)
        elif self.tdi[-1] > 0 and self.di[-1] < 0:
            self.sell(size=1)

class Momentum1DayStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    # -----------------------------------

    def init(self):
        pass  # Pas besoin d'indicateur spécial ici

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand le prix aujourd'hui > prix hier
        if len(self.data) > 1 and self.data.Close[-1] > self.data.Close[-2]:
            self.buy(size=1)

class Momentum5DaysStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    # -----------------------------------

    def init(self):
        pass  # Pas besoin d'indicateur spécial ici

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand le prix aujourd'hui > prix il y a 5 jours
        if len(self.data) > 5 and self.data.Close[-1] > self.data.Close[-6]:
            self.buy(size=1)



/Users/davidzhu/Local/gestion-quantitative/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [26]:
import yfinance as yf
from backtesting import Backtest
import itertools

# 0. Config 
TICKER_LIST = ["AAPL", "MSFT", "GOOGL"]
START_DATE = "2019-01-01"
stats_to_keep = [
    "Start",
    "End",
    "Duration",
    "Equity Final [$]",
    "Return [%]",
    "Return (Ann.) [%]",
    "Volatility (Ann.) [%]",
    "Sharpe Ratio",
    "Sortino Ratio",
    "Max. Drawdown [%]",
    "# Trades",
    "Win Rate [%]",
    "Profit Factor",
    "Expectancy [%]"
]
strategy_list = [
    RSIStrategy,
    BollingerStrategy,
    # ChaikinMoneyFlowStrategy,
    StochasticRStrategy,
    TDIStrategy,
    Momentum1DayStrategy,
    Momentum5DaysStrategy
]
hold_bar_list = [1, 5]

for TICKER in TICKER_LIST:
    # 1. Download data
    data = yf.download(TICKER, start=START_DATE, progress=False)[
                ["Open", "High", "Low", "Close", "Volume"]
            ].dropna().droplevel(1, axis=1)  # remove multi-index
    data.index.name = "Date"


    # 2. Create directories for results
    import os 
    path = os.getcwd()
    html_results_path = os.path.dirname(path) + "/data/html_results/"
    data_results_path = os.path.dirname(path) + "/data/csv_results/"
    os.makedirs(html_results_path, exist_ok=True)
    os.makedirs(data_results_path, exist_ok=True)

    # 3. Run backtests for each strategy and hold_bar combination
    for strategy, hold_bar in list(itertools.product(strategy_list, hold_bar_list)):

        print(f"Running backtest with {TICKER} data...")
        print(f"Config : {strategy.__name__} with hold_bars={hold_bar}")

        strategy.hold_bars = hold_bar
        bt = Backtest(data, strategy,
                        cash=10_000, commission=0.002, trade_on_close=True)
        stats = bt.run()
        
        # Save results
        filename = f"{TICKER}_{strategy.__name__}_holdbars{hold_bar}"
        # Save plot
        bt.plot(plot_width=1200, 
                filename=html_results_path + filename + ".html",
                open_browser=False)
        # Save trades
        stats._trades.to_csv(data_results_path + filename + "_trades.csv", index=True)
        # Save stats
        stats_df = stats[stats_to_keep]
        stats_df.to_csv(data_results_path + filename + "_stats.csv", index=True)


Running backtest for RSIStrategy with hold_bars=1...
Running backtest for RSIStrategy with hold_bars=5...
Running backtest for BollingerStrategy with hold_bars=1...
Running backtest for BollingerStrategy with hold_bars=5...
Running backtest for StochasticRStrategy with hold_bars=1...
Running backtest for StochasticRStrategy with hold_bars=5...
Running backtest for TDIStrategy with hold_bars=1...
Running backtest for TDIStrategy with hold_bars=5...
Running backtest for Momentum1DayStrategy with hold_bars=1...
Running backtest for Momentum1DayStrategy with hold_bars=5...
Running backtest for Momentum5DaysStrategy with hold_bars=1...
Running backtest for Momentum5DaysStrategy with hold_bars=5...
Running backtest for RSIStrategy with hold_bars=1...
Running backtest for RSIStrategy with hold_bars=5...
Running backtest for BollingerStrategy with hold_bars=1...
Running backtest for BollingerStrategy with hold_bars=5...
Running backtest for StochasticRStrategy with hold_bars=1...
Running backt